In [3]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
file_path = '/content/drive/MyDrive/csv/WELFake_Dataset.csv'
data = pd.read_csv(file_path)
data.head()


,`1`,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [6]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=7693a3ce39437c56f2f764e2947d77008a9efe93694794e28e196c2b6bd9634f
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [8]:

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    try:
        text = re.sub(r'[^a-zA-Z]', ' ', text)
        tokens = word_tokenize(text)
        tokens = [word.lower() for word in tokens]
        tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    except TypeError:
        return ''


In [9]:
import swifter
data['clean_text'] = data['text'].swifter.apply(preprocess_text)
data['clean_title_text'] = data['title'] + " " + data['clean_text']


Pandas Apply:   0%|          | 0/72134 [00:00<?, ?it/s]

In [12]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')
nltk.download('punkt')
file_path = '/content/drive/MyDrive/csv/WELFake_Dataset.csv'
data = pd.read_csv(file_path)
data.head()
data['text'] = data['text'].astype(str)
data['title'] = data['title'].astype(str)
data.fillna('', inplace=True)
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)
data['clean_text'] = data['text'].apply(preprocess_text)
data['clean_title'] = data['title'].apply(preprocess_text)
data['clean_title_text'] = data['clean_title'] + " " + data['clean_text']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['clean_title_text'])
sequences = tokenizer.texts_to_sequences(data['clean_title_text'])
padded_sequences = pad_sequences(sequences, maxlen=500)

X = padded_sequences
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=500))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/5
902/902 [==============================] - 2031s 2s/step - loss: 0.2360 - accuracy: 0.9078 - val_loss: 0.1538 - val_accuracy: 0.9435
Epoch 2/5
902/902 [==============================] - 1938s 2s/step - loss: 0.1176 - accuracy: 0.9589 - val_loss: 0.0822 - val_accuracy: 0.9712
Epoch 3/5
902/902 [==============================] - 1943s 2s/step - loss: 0.0824 - accuracy: 0.9726 - val_loss: 0.0796 - val_accuracy: 0.9719
Epoch 4/5
902/902 [==============================] - 1962s 2s/step - loss: 0.0513 - accuracy: 0.9837 - val_loss: 0.0851 - val_accuracy: 0.9729
Epoch 5/5
902/902 [==============================] - 1965s 2s/step - loss: 0.0395 - accuracy: 0.9887 - val_loss: 0.0870 - val_accuracy: 0.9727


In [16]:

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))

model.save('fake_news_classifier.h5')


451/451 [==============================] - 125s 272ms/step
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      7089
           1       0.96      0.99      0.97      7338

    accuracy                           0.97     14427
   macro avg       0.97      0.97      0.97     14427
weighted avg       0.97      0.97      0.97     14427

F1-Score: 0.9735215053763441


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
!pip install flask_ngrok

/bin/bash: line 1: ngrok: command not found


In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

model_path = 'fake_news_classifier.h5'
model = tf.keras.models.load_model(model_path)
tokenizer = Tokenizer(num_words=5000)
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

def predict(news_text):
    clean_news = preprocess_text(news_text)
    seq = tokenizer.texts_to_sequences([clean_news])
    padded = pad_sequences(seq, maxlen=500)
    prediction = (model.predict(padded) > 0.5).astype("int32")[0][0]
    return 'real' if prediction == 1 else 'fake'

if __name__ == '__main__':
    test_articles = [
        "This is a piece of fake news about a celebrity scandal.",
        "The stock market reached an all-time high today due to strong economic growth.",
        "Unbelievable! Aliens have landed on Earth and are communicating with humans."
    ]

    for article in test_articles:
        result = predict(article)
        print(f"Article: {article}\nPrediction: {result}\n")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1/1 [==============================] - 2s 2s/step
Article: This is a piece of fake news about a celebrity scandal.
Prediction: real

1/1 [==============================] - 0s 118ms/step
Article: The stock market reached an all-time high today due to strong economic growth.
Prediction: real

1/1 [==============================] - 0s 116ms/step
Article: Unbelievable! Aliens have landed on Earth and are communicating with humans.
Prediction: real

